In [4]:
import os
import sqlalchemy.orm.decl_api
import time

from api import *
from dataframe import *
from database import *
from file import create_outfile
from unaccent import unaccent

# cfg

In [5]:
cfg = {
    "user": f"{os.environ['POSTGRE_USER']}",
    "password": f"{os.environ['POSTGRE_PASSWORD']}",
    "port": "5432",
    "database": "herbario"
}
filename = "original.csv"
filename_george = "dados-george.csv"

engine, session = connect(cfg)

# Load data

In [6]:
dataframe = pandas.read_csv("original.csv", sep=";", low_memory=False, skipinitialspace=True)
dataframe_george = pandas.read_csv(filename_george, sep=";", low_memory=False, skipinitialspace=True)

In [7]:
create_table_if_not_exists(cfg, engine, "county")
create_table_if_not_exists(cfg, engine, "data")

In [8]:
if session.query(County).count() == 0:
    response = get_municipies()
    for i, county in enumerate(response.json()):
        session.add(create_county(county))
        make_operation(session)


if session.query(DataSP).count() == 0:
    dataframe = preprocess(dataframe)
    for row in dataframe.iterrows():
        session.add(create_datasp(row[1]))
        make_operation(session)


if session.query(DataSP).filter(DataSP.george == True).count() == 0:
    for row in dataframe.iterrows():
        if row[1]["GEORGE"].lower() == "sim":
            session.query(DataSP).filter(DataSP.seq == row[1]["seq"]).update({"george": True}, synchronize_session=False)
            make_operation(session)

In [9]:
def column_is_string_or_varchar_or_text(column):
    return str(column.type).lower() in ("string", "varchar", "text")

def get_columns_text():
    return list([column.key for column in get_columns_table(DataSP) if column_is_string_or_varchar_or_text(column)])


def remove_white_spaces(string):
    return remove_word_started_lowercase(string).split(" ")


def remove_hyphen(string):
    return remove_word_started_lowercase(string).split("-")


def remove_word_started_lowercase(string):
    return re.sub(r"\b[a-z]+\s*", "", string)


def find_and_update(data, list_seq, new_column, value=None):
    for columns in list_of_columns_valid:
        for data_formatted in (remove_white_spaces(data), remove_hyphen(data)):
            if len(data_formatted) > 0:
                q = session.query(DataSP.seq).filter(sqlalchemy.and_(
                    *[sqlalchemy.func.lower(unaccent(getattr(DataSP, columns))).ilike(f"%{s.lower()}%") for s in
                      data_formatted])).all()
            else:
                q = session.query(DataSP.seq).filter(
                    sqlalchemy.func.lower(unaccent(getattr(DataSP, columns))).ilike(f"%{data_formatted}%")).all()
            if len(q) > 0:
                list_seq.append({"seq": remove_set(q), "column": columns, "new_column": new_column, "value_searched": value if value else data})

In [10]:
data_piperaceae = session.query(DataSP).all()
data_uf = session.query(sqlalchemy.func.lower(unaccent(County.uf))).distinct().all()
data_uf_name = session.query(sqlalchemy.func.lower(unaccent(County.uf_name))).distinct().all()
data_county = session.query(unaccent(County.county)).filter(County.uf=="PR").all()

In [11]:
def remove_set(data):
    return list([s for s, in data])

data_uf = remove_set(data_uf)
data_uf_name = remove_set(data_uf_name)
data_county = remove_set(data_county)
data_country = list(["Brazil", "Brasil"])

In [12]:
list_seq = list([])
list_of_columns_valid = get_columns_text()

In [ ]:
start = time.process_time()

for d in data_country:
    find_and_update(d, list_seq, "my_country", value="Brasil")

end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds")

In [13]:
start = time.process_time()

for d in data_uf_name:
    find_and_update(d, list_seq, "my_state")

end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds")

elapsed_time: 520.125 seconds


In [ ]:
start = time.process_time()

for d in data_county:
    find_and_update(d, list_seq, "my_city")

end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds")

In [14]:
for l in list_seq:
    for y in l["seq"]:
        session.query(DataSP).filter(DataSP.seq == y).update(values=dict({l["new_column"]: l["value_searched"]}), synchronize_session=False)
        make_operation(session)

KeyboardInterrupt: 

In [ ]:
create_outfile(list_seq)

In [ ]:
data_has_only_state = session.query(DataSP.seq, DataSP.my_state).filter(sqlalchemy.and_(DataSP.my_state.is_not(None), DataSP.my_country.is_(None))).all()
make_operation(session)

for (seq, state) in sorted(data_has_only_state):
    session.query(DataSP).filter(DataSP.seq == seq).update(values=dict({"my_country": "Brasil"}), synchronize_session=False)
    make_operation(session)

In [ ]:
def get_state_of_city(city):
    state_of_city = session.query(County.uf_name).filter(County.county == city).all()
    make_operation(session)
    return state_of_city

data_has_only_county = session.query(DataSP.seq, DataSP.my_city).filter(sqlalchemy.and_(DataSP.my_city.is_not(None), DataSP.my_state.is_(None), DataSP.my_country.is_(None))).all()
make_operation(session)

print(data_has_only_county)
for (seq, city) in data_has_only_county:
    state_of_city = get_state_of_city(city)
    session.query(DataSP).filter(DataSP.seq == seq).update(values=dict({"my_state": state_of_city, "my_country": "Brasil"}), synchronize_session=False)
    make_operation(session)

In [ ]:
session.close()
engine.dispose()